<a href="https://colab.research.google.com/github/AurelioVill/Aulas-Python/blob/main/Analise_Futebol_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install matplotlib

In [3]:
!pip install mplsoccer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 1.4 MB/s eta 0:00:00


In [4]:
#INSTALAR O SPARK NO COLAB
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=434732770922c3933a9be7046d59740b372d75a78ff7ed2a51563389e145d89f
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [5]:
#IMPORTAR SPARK NO COLAB
from pyspark.sql import SparkSession

In [6]:
#CRIAR SESSÃO SPARK
spark = SparkSession.builder.appName("SessionPySpark").getOrCreate()

In [7]:
#OBTER CONTEXTO SPARK PARA A SESSÃO
sc = spark.sparkContext

In [8]:
from mplsoccer import Pitch, VerticalPitch
import matplotlib.pyplot as plt
import numpy as np

In [9]:
import requests

In [10]:
from bs4 import BeautifulSoup

In [11]:
from pyspark.sql.functions import col

In [12]:
import pandas as pd
from pyspark.sql import SparkSession

# Crie uma sessão do Spark
spark = SparkSession.builder.appName("My App").getOrCreate()

# Leia a tabela HTML com pandas
df_pandas = pd.read_html("https://fbref.com/pt/partidas/d3926429/Corinthians-Atletico-Mineiro-2024Abril14-Serie-A", attrs={'id':'stats_bf4acd28_possession'})[0]

# Achatamento da hierarquia de colunas
df_pandas.columns = ['_'.join(col).strip() for col in df_pandas.columns.values]

# Renomeando as colunas
df_pandas.columns = [col.replace(' ', '_') for col in df_pandas.columns]

# Criando uma coluna index,a coluna index não existe no DataFrame df_spark. Isso porque o método pd.read_html não cria uma coluna index automaticamente.
df_pandas.reset_index(inplace=True)

# Converta o DataFrame do pandas para um DataFrame do Spark
df_spark = spark.createDataFrame(df_pandas)

# Remova a linha com índice 16
df_estatisticas = df_spark.filter(col("index") != 16)

# Exiba o resultado
df_estatisticas.show()

+-----+--------------------------+--------------------+------------------------+-----------------------+------------------------+-----------------------+-----------------+----------------+------------------+----------------------+------------------------+----------------+----------------+------------+-----------+------------+------------+-------------+-------------------+-----------------+-----------------+--------------+-------------+-------------+--------------------------+-------------+------------+-------------+
|index|Unnamed:_0_level_0_Jogador|Unnamed:_1_level_0_#|Unnamed:_2_level_0_Nação|Unnamed:_3_level_0_Pos.|Unnamed:_4_level_0_Idade|Unnamed:_5_level_0_Min.|Contatos_Contatos|Contatos_Def_Pen|Contatos_Terço_Def|Contatos_Terço_Central|Contatos_Terço_de_Ataque|Contatos_Att_Pen|Contatos_Em_jogo|Dribles_Tent|Dribles_Suc|Dribles_Suc%|Dribles_Tkld|Dribles_Tkld%|Conduções_Conduções|Conduções_DistTot|Conduções_DistPrg|Conduções_PrgC|Conduções_1/3|Conduções_CPA|Conduções_Perda_de_Domíni

In [13]:
df_estatisticas.printSchema()

root
 |-- index: long (nullable = true)
 |-- Unnamed:_0_level_0_Jogador: string (nullable = true)
 |-- Unnamed:_1_level_0_#: double (nullable = true)
 |-- Unnamed:_2_level_0_Nação: string (nullable = true)
 |-- Unnamed:_3_level_0_Pos.: string (nullable = true)
 |-- Unnamed:_4_level_0_Idade: string (nullable = true)
 |-- Unnamed:_5_level_0_Min.: long (nullable = true)
 |-- Contatos_Contatos: long (nullable = true)
 |-- Contatos_Def_Pen: long (nullable = true)
 |-- Contatos_Terço_Def: long (nullable = true)
 |-- Contatos_Terço_Central: long (nullable = true)
 |-- Contatos_Terço_de_Ataque: long (nullable = true)
 |-- Contatos_Att_Pen: long (nullable = true)
 |-- Contatos_Em_jogo: long (nullable = true)
 |-- Dribles_Tent: long (nullable = true)
 |-- Dribles_Suc: long (nullable = true)
 |-- Dribles_Suc%: double (nullable = true)
 |-- Dribles_Tkld: long (nullable = true)
 |-- Dribles_Tkld%: double (nullable = true)
 |-- Conduções_Conduções: long (nullable = true)
 |-- Conduções_DistTot: long

In [14]:
# Criar uma sessão Spark
spark = SparkSession.builder.appName("Futebol").getOrCreate()

# Criar DataFrames a partir dos dicionários
posicionamento_df = spark.createDataFrame([
    ("Goleiro", 2, 40),
    ("Ataque", 90, 40),
    ("Meio-campista-esquerda", 85, 15),
    ("Meio-campista-direita", 85, 65),
    ("Meio-campista-meia", 70, 40),
    ("Meio-defensor-direita", 44, 60),
    ("Meio-defensor-centro", 44, 40),
    ("Meio-defensor-esquerda", 44, 20),
    ("Defesa-centro", 30, 40),
    ("Defesa-esquerda", 24, 30),
    ("Defesa-direita", 24, 50),
    ("Lateral-esquerdo", 55, 10),
    ("Lateral-direito", 55, 70)
], ["posicao", "y", "x"])

jogadores_df = spark.createDataFrame([
    ("Yuri Alberto", "Ataque"),
    ("Wesley Ribeiro", "Meio-campista-esquerda"),
    ("Ángel Romero", "Meio-campista-direita"),
    ("Pedro Raul", "Meio-campista-direita"),
    ("Rodrigo Garro", "Meio-campista-meia"),
    ("Igor Coronado", "Meio-campista-meia"),
    ("Fausto Vera", "Meio-defensor-direita"),
    ("Maycon", "Meio-defensor-direita"),
    ("Raniele", "Meio-defensor-esquerda"),
    ("Hugo", "Defesa-direita"),
    ("Gustavo Henrique", "Defesa-centro"),
    ("Paulinho", "Meio-campista-meia"),
    ("Félix Torres Caicedo", "Defesa-esquerda"),
    ("Fagner", "Lateral-esquerdo"),
    ("Matheuzinho", "Lateral-direito"),
    ("Cássio", "Goleiro")
], ["nome", "posicao"])

In [32]:
# Mesclar os dois DataFrames na coluna "posicao"
df_merge = posicionamento_df.join(jogadores_df, posicionamento_df.posicao == jogadores_df.posicao, "left_outer")


In [33]:
df_merge.show()

+--------------------+---+---+--------------------+--------------------+
|             posicao|  y|  x|                nome|             posicao|
+--------------------+---+---+--------------------+--------------------+
|              Ataque| 90| 40|        Yuri Alberto|              Ataque|
|Meio-campista-dir...| 85| 65|          Pedro Raul|Meio-campista-dir...|
|Meio-campista-dir...| 85| 65|        Ángel Romero|Meio-campista-dir...|
|Meio-campista-esq...| 85| 15|      Wesley Ribeiro|Meio-campista-esq...|
|             Goleiro|  2| 40|              Cássio|             Goleiro|
|  Meio-campista-meia| 70| 40|            Paulinho|  Meio-campista-meia|
|  Meio-campista-meia| 70| 40|       Igor Coronado|  Meio-campista-meia|
|  Meio-campista-meia| 70| 40|       Rodrigo Garro|  Meio-campista-meia|
|Meio-defensor-dir...| 44| 60|              Maycon|Meio-defensor-dir...|
|Meio-defensor-dir...| 44| 60|         Fausto Vera|Meio-defensor-dir...|
|Meio-defensor-centro| 44| 40|                NULL|

In [27]:
df_merge.printSchema()

root
 |-- posicao: string (nullable = true)
 |-- y: long (nullable = true)
 |-- x: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- posicao: string (nullable = true)



In [29]:
display(df_merge.columns)

['posicao', 'y', 'x', 'nome', 'posicao']

In [17]:
df_estatisticas.show()

+-----+--------------------------+--------------------+------------------------+-----------------------+------------------------+-----------------------+-----------------+----------------+------------------+----------------------+------------------------+----------------+----------------+------------+-----------+------------+------------+-------------+-------------------+-----------------+-----------------+--------------+-------------+-------------+--------------------------+-------------+------------+-------------+
|index|Unnamed:_0_level_0_Jogador|Unnamed:_1_level_0_#|Unnamed:_2_level_0_Nação|Unnamed:_3_level_0_Pos.|Unnamed:_4_level_0_Idade|Unnamed:_5_level_0_Min.|Contatos_Contatos|Contatos_Def_Pen|Contatos_Terço_Def|Contatos_Terço_Central|Contatos_Terço_de_Ataque|Contatos_Att_Pen|Contatos_Em_jogo|Dribles_Tent|Dribles_Suc|Dribles_Suc%|Dribles_Tkld|Dribles_Tkld%|Conduções_Conduções|Conduções_DistTot|Conduções_DistPrg|Conduções_PrgC|Conduções_1/3|Conduções_CPA|Conduções_Perda_de_Domíni

In [18]:
from pyspark.sql.functions import col, sum, round

In [19]:
from pyspark.sql import Window

especificacao_janela = Window.partitionBy().rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, round

# Crie uma sessão Spark
spark = SparkSession.builder.appName("Estatisticas").getOrCreate()

In [21]:
df_estatisticas = df_estatisticas.withColumnRenamed('Unnamed:_0_level_0_Jogador', 'Jogador')

In [22]:
df_estatisticas = df_estatisticas.withColumnRenamed('Unnamed:_5_level_0_Min.', 'Minutos')

In [23]:
df_estatisticas.printSchema()

root
 |-- index: long (nullable = true)
 |-- Jogador: string (nullable = true)
 |-- Unnamed:_1_level_0_#: double (nullable = true)
 |-- Unnamed:_2_level_0_Nação: string (nullable = true)
 |-- Unnamed:_3_level_0_Pos.: string (nullable = true)
 |-- Unnamed:_4_level_0_Idade: string (nullable = true)
 |-- Minutos: long (nullable = true)
 |-- Contatos_Contatos: long (nullable = true)
 |-- Contatos_Def_Pen: long (nullable = true)
 |-- Contatos_Terço_Def: long (nullable = true)
 |-- Contatos_Terço_Central: long (nullable = true)
 |-- Contatos_Terço_de_Ataque: long (nullable = true)
 |-- Contatos_Att_Pen: long (nullable = true)
 |-- Contatos_Em_jogo: long (nullable = true)
 |-- Dribles_Tent: long (nullable = true)
 |-- Dribles_Suc: long (nullable = true)
 |-- Dribles_Suc%: double (nullable = true)
 |-- Dribles_Tkld: long (nullable = true)
 |-- Dribles_Tkld%: double (nullable = true)
 |-- Conduções_Conduções: long (nullable = true)
 |-- Conduções_DistTot: long (nullable = true)
 |-- Conduções_D

In [24]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Definir a janela global
global_window = Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

# Renomear colunas problemáticas para evitar caracteres especiais
df_estatisticas = df_estatisticas.withColumnRenamed('Conduções_Conduções', 'Conducoes_Conducoes')
df_estatisticas = df_estatisticas.withColumnRenamed('Conduções_DistTot', 'Conducoes_DistTot')
df_estatisticas = df_estatisticas.withColumnRenamed('Conduções_DistPrg', 'Conducoes_DistPrg')
df_estatisticas = df_estatisticas.withColumnRenamed('Conduções_PrgC', 'Conducoes_Prgc')
df_estatisticas = df_estatisticas.withColumnRenamed('Conduções_Perda_de_Domínio', 'Conducoes_Perda_Dominio')
df_estatisticas = df_estatisticas.withColumnRenamed('Conduções_Dis', 'Conducoes_Dis')

# Calcular as porcentagens e arredondar para duas casas decimais
df_estatisticas = (
    df_estatisticas
    .withColumn(
        '%Conducoes',
        (F.col('Conducoes_Conducoes') / F.sum('Conducoes_Conducoes').over(global_window)).cast('decimal(10,2)')
    )
    .withColumn(
        '%DistTotal',
        (F.col('Conducoes_DistTot') / F.sum('Conducoes_DistTot').over(global_window)).cast('decimal(10,2)')
    )
    .withColumn(
        '%DistAtaque',
        (F.col('Conducoes_DistPrg') / F.sum('Conducoes_DistPrg').over(global_window)).cast('decimal(10,2)')
    )
    .withColumn(
        '%ProjAtaque9m',
        (F.col('Conducoes_Prgc') / F.sum('Conducoes_Prgc').over(global_window)).cast('decimal(10,2)')
    )
    .withColumn(
        '%PerdaDomínio',
        (F.col('Conducoes_Perda_Dominio') / F.sum('Conducoes_Perda_Dominio').over(global_window)).cast('decimal(10,2)')
    )
    .withColumn(
        '%Desarmes',
        (F.col('Conducoes_Dis') / F.sum('Conducoes_Dis').over(global_window)).cast('decimal(10,2)')
    )
    .withColumn(
        '%minutos',
        (F.col('Minutos') / 90 * 100).cast('decimal(10,2)')
    )
)

# Mesclar com o DataFrame df_jogadores
df_estatisticas = df_estatisticas.join(
    jogadores_df,
    df_estatisticas['Jogador'] == jogadores_df['nome'],
    how='inner'
).drop(jogadores_df['nome'])

# Mostrar o resultado para verificação
df_estatisticas.show()




+-----+--------------------+--------------------+------------------------+-----------------------+------------------------+-------+-----------------+----------------+------------------+----------------------+------------------------+----------------+----------------+------------+-----------+------------+------------+-------------+-------------------+-----------------+-----------------+--------------+-------------+-------------+-----------------------+-------------+------------+-------------+----------+----------+-----------+-------------+-------------+---------+--------+--------------------+
|index|             Jogador|Unnamed:_1_level_0_#|Unnamed:_2_level_0_Nação|Unnamed:_3_level_0_Pos.|Unnamed:_4_level_0_Idade|Minutos|Contatos_Contatos|Contatos_Def_Pen|Contatos_Terço_Def|Contatos_Terço_Central|Contatos_Terço_de_Ataque|Contatos_Att_Pen|Contatos_Em_jogo|Dribles_Tent|Dribles_Suc|Dribles_Suc%|Dribles_Tkld|Dribles_Tkld%|Conducoes_Conducoes|Conducoes_DistTot|Conducoes_DistPrg|Conducoes_Prg

In [25]:
df_estatisticas.columns

['index',
 'Jogador',
 'Unnamed:_1_level_0_#',
 'Unnamed:_2_level_0_Nação',
 'Unnamed:_3_level_0_Pos.',
 'Unnamed:_4_level_0_Idade',
 'Minutos',
 'Contatos_Contatos',
 'Contatos_Def_Pen',
 'Contatos_Terço_Def',
 'Contatos_Terço_Central',
 'Contatos_Terço_de_Ataque',
 'Contatos_Att_Pen',
 'Contatos_Em_jogo',
 'Dribles_Tent',
 'Dribles_Suc',
 'Dribles_Suc%',
 'Dribles_Tkld',
 'Dribles_Tkld%',
 'Conducoes_Conducoes',
 'Conducoes_DistTot',
 'Conducoes_DistPrg',
 'Conducoes_Prgc',
 'Conduções_1/3',
 'Conduções_CPA',
 'Conducoes_Perda_Dominio',
 'Conducoes_Dis',
 'Recepção_Rec',
 'Recepção_PrgR',
 '%Conducoes',
 '%DistTotal',
 '%DistAtaque',
 '%ProjAtaque9m',
 '%PerdaDomínio',
 '%Desarmes',
 '%minutos',
 'posicao']

In [26]:
from pyspark.sql import functions as F

df_estatisticas = df_estatisticas.withColumn('inicio_ataque_x', F.col("x"))
df_estatisticas = df_estatisticas.withColumn('inicio_ataque_y', F.col("y"))
df_estatisticas = df_estatisticas.withColumn('fim_ataque_y', F.col("y") + (F.col('%DistAtaque') * 120) / 100)
df_estatisticas = df_estatisticas.withColumn('fim_ataque_x', F.lit(40))

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `x` cannot be resolved. Did you mean one of the following? [`index`, `Jogador`, `Minutos`, `posicao`, `%minutos`].;
'Project [index#0L, Jogador#333, Unnamed:_1_level_0_##2, Unnamed:_2_level_0_Nação#3, Unnamed:_3_level_0_Pos.#4, Unnamed:_4_level_0_Idade#5, Minutos#363L, Contatos_Contatos#7L, Contatos_Def_Pen#8L, Contatos_Terço_Def#9L, Contatos_Terço_Central#10L, Contatos_Terço_de_Ataque#11L, Contatos_Att_Pen#12L, Contatos_Em_jogo#13L, Dribles_Tent#14L, Dribles_Suc#15L, Dribles_Suc%#16, Dribles_Tkld#17L, Dribles_Tkld%#18, Conducoes_Conducoes#393L, Conducoes_DistTot#423L, Conducoes_DistPrg#453L, Conducoes_Prgc#483L, Conduções_1/3#23L, ... 14 more fields]
+- Project [index#0L, Jogador#333, Unnamed:_1_level_0_##2, Unnamed:_2_level_0_Nação#3, Unnamed:_3_level_0_Pos.#4, Unnamed:_4_level_0_Idade#5, Minutos#363L, Contatos_Contatos#7L, Contatos_Def_Pen#8L, Contatos_Terço_Def#9L, Contatos_Terço_Central